In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'
import time
import jax 
import pickle
import matplotlib.pyplot as plt
from config import *
from phyOT.utils import *
from phyOT.utils import *

In [ ]:
def flush(u):
    return jnp.where(jnp.abs(u)<1e-8, 0, u)

In [ ]:
file = 'data/data.pkl'

with open(file, 'rb') as f:
    data = pickle.load(f)

ell = 5
idx = [2, 3, 4]
for ix in idx:
    plt.contourf(data['grid'][0], data['grid'][1], flush(data['dataUs'][ix]), 50)
    plt.colorbar()
    plt.scatter(data['obs_locs'][:, 0], data['obs_locs'][:, 1], c='k', s=5)
    plt.xlabel(r'$x_{(1)}$')
    plt.ylabel(r'$x_{(2)}$')
    plt.savefig(f'data/plots/sln_obs_{ix}.png')
    plt.show()

    plt.contourf(data['grid'][0], data['grid'][1], flush(data['dataZs'][ix]), 50)
    plt.colorbar()
    plt.xlabel(r'$x_{(1)}$')
    plt.ylabel(r'$x_{(2)}$')
    plt.savefig(f'data/plots/z_field_{ix}.png')
    plt.show()

    plt.contourf(data['grid'][0], data['grid'][1], flush(data['dataAs'][ix]), 50, cmap='plasma')
    plt.colorbar()
    plt.xlabel(r'$x_{(1)}$')
    plt.ylabel(r'$x_{(2)}$')
    plt.savefig(f'data/plots/z_field_{ix}.png')
    plt.show()
    
    # a_field = data['dataAs'][ix]
    # dx = 1. / (solver.grid[0].shape[0] - 1)
    # a_L2 = jnp.sqrt( jnp.sum( a_field**2. * dx**2. ) )
    # print(f'a_L2: {a_L2}')
    # a_norm = a_L2
    # a_bar_field = a_field / a_norm
    # kappa_1, kappa_2 = 1., 2.
    # z_field = jnp.tanh((a_bar_field) * ell ) * 0.5 * (kappa_2 - kappa_1) + kappa_1 + 0.5 * (kappa_2 - kappa_1)
    # plt.contourf(data['grid'][0], data['grid'][1], flush(z_field), 50)
    # plt.colorbar()
    # plt.xlabel(r'$x_{(1)}$')
    # plt.ylabel(r'$x_{(2)}$')
    # plt.savefig(f'data/plots/z_field_smooth_{ix}.png')
    # plt.show()


In [ ]:
with open('data/model.pkl', 'rb') as f:
    model = pickle.load(f)

loss = model['loss']
lossres = model['aux']['aux'][:,2]

plt.semilogy(loss, label=r'$J_1$')
plt.semilogy(lossres, label=r'$J_2$')
plt.legend()
plt.grid()
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.savefig('data/plots/plot_loss_model.pdf')
plt.show()
lw = 1
w_avg = 5_000

aux = model['aux']
sigma_val = jnp.exp(aux['sigmas'])
ell_val   = jnp.exp( aux['ells'] )
nu_val    = jnp.exp(aux['nus'])


plt.plot(nu_val, c = 'tab:blue', label=r'$\nu$')
plt.plot(nu_val**0. * 1.5, '--', c='k', label=r'true $\nu$')
plt.plot(ell_val, c='tab:orange', label=r'$\ell$')
plt.plot(ell_val**0. * 0.5, ':', c='k', label=r'true $\ell$')
plt.grid()
plt.xlabel('Iterations')
plt.ylabel(r'$\nu, \ell$')

plt.legend(loc='right')

plt.savefig('data/plots/plot_convergence_model.pdf')
plt.show()

print('run time', model['time']/60, 'mins')

print('error nu', 100*jnp.linalg.norm(nu_val[-1] - 1.5)/(jnp.linalg.norm(1.5)))
print('error ell', 100*jnp.linalg.norm(ell_val[-1] - 0.5)/(jnp.linalg.norm(0.5)))
print('error sig', 100*jnp.linalg.norm(sigma_val[-1] - 1.)/(np.linalg.norm(1.)))
